# German Credit Risk 

Notebook using German Credit Risk Dataset from https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data and methods from paper: "Explainable AI for Credit Assessment in Banks" (https://www.mdpi.com/1911-8074/15/12/556).

Note that text for explanation in this notebook is for the most part copied/adapted from the paper above.